In [ ]:
import pathlib
import sys
import warnings

import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import autocrit

In [ ]:
sys.path.append("..")

import shared.format
import shared.tools

In [ ]:
plt.rcParams["font.size"] = shared.format.FONTSIZE

In [ ]:
pd.options.mode.chained_assignment = None

warnings.filterwarnings(
    "ignore", category=FutureWarning,
    message="Using a non-tuple sequence for multidimensional indexing is deprecated;",
    module="autograd")

In [ ]:
import autocrit

In [ ]:
import autocrit_tools
import autocrit_tools.linearpaper.utils as utils

In [ ]:
pickle_dir = pathlib.Path("data") / "linear-results"

In [ ]:
def compute_index(spectrum, eigval_cutoff=1e-5):
    return np.mean(spectrum < -eigval_cutoff)

In [ ]:
cutoffs = [1e-10, 1e-6, 1e-4, np.inf]
cutoff_strings = [r"$\varepsilon=$1e-10", r"$\varepsilon=$1e-6", r"$\varepsilon=$1e-4", "no cutoff"]

In [ ]:
cutoff_fnames = ["1e-10", "1e-6", "1e-4", "none"]

cutoff_cp_df_names= [pickle_dir / ("cutoff_" + cutoff_fname + ".pkl")
                     for cutoff_fname in cutoff_fnames]

In [ ]:
import autocrit_tools.path as path
import autocrit_tools.dataframes as dataframes

In [ ]:
def clean_path_columns(df, path_columns, split_str, new_prefix):
    for path_column in path_columns:
        df[path_column] = df[path_column].apply(clean_path, args=(split_str, new_prefix))
        
def clean_path(path, split_str, new_prefix):
    path_end = path.split(split_str)[-1]
    return os.path.join(new_prefix, path_end)

def add_index_sets(cp_df):
    maps = cp_df.final_theta.apply(
        lambda theta: utils.theta_to_map(theta, NETWORK))
    index_sets = [utils.map_to_index_set(_map) for _map in maps]

    cp_df["index_set"] = index_sets

In [ ]:
root_dir = "data/"

In [ ]:
data_ID = "linear-results"
network_ID = "dlae"

network_paths = path.ExperimentPaths(data_ID=data_ID, network_ID=network_ID, root=root_dir)

target_optimization_ID = "gd_optimizer_fimeXE"

In [ ]:
optimization_df = dataframes.construct_experiments_df(
    network_paths.network_dir)

target_optimization_row = optimization_df.loc[target_optimization_ID]

data, NETWORK, _ = dataframes.reconstruct_from_row(
    target_optimization_row)

In [ ]:
optimizer_paths = path.ExperimentPaths.from_optimizer_dir(target_optimization_row.optimizer_dir)

In [ ]:
critfinders_dir = optimizer_paths.optimizer_dir

cf_df = dataframes.construct_experiments_df(
    critfinders_dir, experiment_type="critfinder")

In [ ]:
expanded_cf_df = pd.DataFrame.from_dict(
    dict(cf_df.finder_kwargs_finder)
    ).transpose().join(cf_df)

In [ ]:
def select_critfinders(cf_df, method_str, theta_perturbs=None, init_theta="uniform_f", 
                       min_runs=0, max_runs=1000):
    
    selector = cf_df.index.str.startswith(method_str) &\
               cf_df.init_theta_experiment.str.endswith(init_theta)
    
    if theta_perturbs is None:
        selector = selector & cf_df.theta_perturb_experiment.isnull()
    else:
        selector = selector & cf_df.theta_perturb_experiment.isin(theta_perturbs)
    
    cf_ids = expanded_cf_df.index[selector]
    
    cp_dfs =  utils.make_cp_dfs(cf_ids, cf_df)
    
    try:
        cp_dfs, cf_ids = zip(*[(cp_df, cf_id) for cp_df, cf_id in zip(cp_dfs, cf_ids)
                               if (len(cp_df) >= min_runs) & (len(cp_df) <= max_runs)])
    except ValueError:
        cp_dfs = cf_ids = []
    
    
    return cp_dfs, list(cf_ids)

In [ ]:
def to_candidate_cp(cp_df, network, return_failures=False, cp_cutoff=1e-10):
    
    if return_failures:
        candidate_cp_df, failures_df = utils.filter_to_candidate_cps(
            cp_df, network, cp_cutoff=cp_cutoff, return_failures=True)
        candidate_cp_df["morse_index"] = utils.get_hessian_info(
            candidate_cp_df.final_theta, network)[-1]
        return candidate_cp_df, failures_df
    else:
        candidate_cp_df = utils.filter_to_candidate_cps(
            cp_df, network, cp_cutoff=cp_cutoff, return_failures=False)
        candidate_cp_df["morse_index"] = utils.get_hessian_info(
            candidate_cp_df.final_theta, network)[-1]
        
        return candidate_cp_df

In [ ]:
MR_cp_dfs, MR_cf_ids = select_critfinders(expanded_cf_df,
                                          method_str="newtonMR",
                                         theta_perturbs=[None, -2])

In [ ]:
merged_MR_cp_df = pd.concat(MR_cp_dfs)

In [ ]:
cutoff_cp_dfs = [make_cutoff_cp_df(merged_MR_cp_df, NETWORK, cutoff) for cutoff in cutoffs]

In [ ]:
merged_MR_candidate_cp_df, merged_MR_failures_df = to_candidate_cp(
    merged_MR_cp_df, NETWORK,
    return_failures=True, cp_cutoff=1e-10)

In [ ]:
def make_slim(cp_df):
    slim_df = cp_df.drop(
        axis="columns",
        labels= ["log_kwargs", "alpha", "beta", "rho", "check_pure", "rho_pure",
        "log_mrqlp", "rtol", "maxit", "thetas", "run_length", "alphas",
        "pure_accepted"])
    return slim_df

In [ ]:
slim_cutoff_cp_dfs = [make_slim(cutoff_cp_df) for cutoff_cp_df in cutoff_cp_dfs]

In [ ]:
[cutoff_cp_df.to_pickle(cutoff_cp_df_name)
 for cutoff_cp_df, cutoff_cp_df_name in zip(cutoff_cp_dfs,cutoff_cp_df_names)];

In [ ]:
slim_MR_df.to_pickle("data/linear-results/newtonMR.pkl")

In [ ]:
def make_cutoff_cp_df(cp_df, network, cp_cutoff):
    cutoff_cp_df, failures_df = utils.filter_to_candidate_cps(
        cp_df, network, cp_cutoff=cp_cutoff, return_failures=True, cut_early=True)
    cutoff_cp_df["morse_index"] = utils.get_hessian_info(
        cutoff_cp_df.candidate_theta, network)[-1]
    return cutoff_cp_df

In [ ]:
import autocrit_tools.linearpaper.panels as panels

In [ ]:
panels.make_trajectory_panel(merged_MR_candidate_cp_df,
                             merged_MR_failures_df);
plt.xscale("linear"); plt.xlim(0, 500);

In [ ]:
panels.make_loss_index_panel(merged_MR_candidate_cp_df,
                             analytical_cp_df);